In [ ]:
#what this file does: 

#1. Takes terms that are already parsed by json_parse_script.ipynb#2. make everything lower case; get rid of extra characters; ensure certain words like azure and office is followed by a space or a comma
#3. put all skill in row (uniques) with counts and users with that skill
#4. run spell check on uniques terms 

initial py speller: 33 sec for every 100 rows. 
After splitting, 38 sec for every 1000 rows. 

after spell checking on keywords, there were 200 less unique skills than before (original uniques =7,890 new= 7,632)
after spell checking on recruit, improved by 69 less unique skills (original = 2469 new = 2403)
after spell checking on ask me about, improved by x (original = 4459 new = 4305)

In [ ]:
pip install cython
pip install hdbscan
pip install pandas
pip install BERTopic
pip install pyspellchecker

import hdbscan
from sklearn.datasets import make_blobs

In [ ]:
import json
import pandas as pd
import re
from bertopic import BERTopic
from spellchecker import SpellChecker
import pickle


In [ ]:
#Import files already parsed: advisors/skills, projects/keywords/recruiting

skills_file = pd.read_csv('csv_file)


#split files into data frames. Rename columns on import.
skills = pd.DataFrame(data = skills_file, columns =['alias', 'C'])
skills.rename(columns = {"alias":"user",
                "C":"skills"}, inplace = True)




In [ ]:
# modify input_file variable to select file to be cleaned
input_data = pd.DataFrame(skills_file)

#column_to_clean = column to be cleaned
column_to_clean = skills_file['skills']

#df = file to be cleaned
# df = pd.DataFrame(input_file)
input_data

In [ ]:
input_data['Cleaned'] = clean_column(column_to_clean)
input_data

In [ ]:
#take input data, split apart, put unique elements into list, count how many entities have that element, ie column 1 = azure, column 2 = user1, user2... then run spell check on it.

dfx = process_skills_data(input_data)
dfx

In [ ]:
#run spell check by itself on the terms

newdf = dfx.drop(['user'], axis = 1)

y = spell_check_column(newdf)

In [ ]:
#save the old terms and the corrrected terms to a csv

y.to_csv('skills_spell_checked.csv')

In [ ]:
#use this function to check in after each improvement to the code to understand how many unique skills there are - goal of spell check and reg ex expressions in clean column 
#was to reduce the amount of skills/terms as a whole. 

z_count = pd.unique(y['Spelled'])
z_unique = 0

for item in z_count:
    z_unique += 1

print(z_unique)

In [ ]:
# Function to clean: extra characters, spaces, specific words etc

def clean_column(column):
    # Convert column to lowercase
    column = column.str.lower()
    
    # Remove - and |
    column = column.apply(lambda x: re.sub(r'[-|\||/|"|“|”|#\@|(|)]', ' ', str(x)))

    # Remove any instance of two in a row of . & % and any double space
    column = column.apply(lambda x: re.sub(r'[\.\&\%]{2,}|\s{2,}]', ' ', str(x)))
    
    #replace  all !, ', and $ within blank.
    column = column.apply(lambda x: re.sub(r'[!\'\$]', '', str(x)))

    #replace any double space up to 5 spaces with a comma and one space
    column = column.apply(lambda x: re.sub(r'([ ]{2,5})', ', ', str(x)))

    # Remove &, and, replace with ','
    column = column.apply(lambda x: re.sub(r'&|and', ',', str(x)))

    #if azure or office has a letter right after or before, insert a space. 
    column = column.apply(lambda x: re.sub(r'[a-zA-Z]+azure|azure[a-zA-Z]+', ' ', str(x)))

    column = column.apply(lambda x: re.sub(r'[a-zA-Z]+office|office[a-zA-Z]+', ' ', str(x)))
    
    return column

In [ ]:
# the below function takes the output Cleaned column from the function clean_column and modifies the data to have the skills be unique as rows.
#  it also counts how many users are list that skill and then lists those users

#Variables: 
#df is file to be cleaned, assigned above. 
#Cleaned = column processed in previous step (removing &'s etc)
#columns lose name after 1st step, so columns need to be renamed. Cannot delete names in step 2

def process_skills_data(df):
    #1. set user info as index, access cleaned column and split column info on ,, then expand into seperate columns, 
    # then put columns into rows. Level column created in this step indicates number/place within string. IE hannah: skills: ai, ml level 2 = ml
    df1 = df.set_index("user").Cleaned.str.split(',', expand=True).stack().reset_index()

    # 2. Rename the columns for clarity
    df1.columns = ["user", "level", "Cleaned"]

    # 3. Remove any whitespace around the skill name in column cleaned
    df1["Cleaned"] = df1["Cleaned"].str.strip()

    # 4. Group by the skill name in column Cleaned and find all unique users with that skill
    result = df1.groupby("Cleaned").user.unique()
    

    # 5. CleanCleanColumnolumnonvert the result to a dataframe and add a count column for the number of users with each skill
    result = result.to_frame().reset_index()

    #this takes a blank string, converts to NA then drops it from data frame. 
    result.replace(r'^s*$', float('NaN'), regex = True, inplace = True)
    result.dropna(inplace = True) 

    #this line counts how many users listed that skill.
    result["Count"] = result["user"].apply(lambda x: len(x))

    

    return result



In [ ]:
# take a column,
# split it into elements, 
# then run spell check on those Elements
# put correct elements in seperate column, do nothing if word is spelled correctly 
import time

def spell_check_column(df):
    spell = SpellChecker()
    spell.word_frequency.load_text_file('known_words.txt')

    def apply_spell_correction(wordList):
        updatedWordList = []
        words = wordList[0].split()

        for word in words:
            if len(word) > 3:
                newWord = spell.correction(word)
                if type(newWord) == type(None):
                    updatedWordList.append(word)
                else:
                    updatedWordList.append(newWord)    
            else: 
                updatedWordList.append(word)
        return ' '.join(updatedWordList)
    

    df['Spelled'] = df['Cleaned'].astype(str)

    df['Spelled'] = df['Spelled'].str.split(',', expand = False)

    df['Spelled'] = df['Spelled'].apply(apply_spell_correction)
    
   
    
    result = df
    return(result)
 

Begin topicizing elements here. 

In [ ]:
#read csv of spell checked elements that you want to topic ize

df1 = pd.read_csv('spell_checked_terms.csv')

df = df1

In [ ]:
#load data to df, convert all elements to string.

df = pd.DataFrame(data = df2)
df = df[['Spelled']].astype(str)
df

In [ ]:
#take df, convert column to be bucketized to list. 

data = df
docs = data["Spelled"].to_list()

#suggest topics to BerTopic to nudge it in the right direction
seed_topic_list = [["azure", "m365", "email", "dynamics", 
                "security"]]


#Give model your parametersVerbose= explain step by step to developer. nr topics means only give me this many topics 
topic_model = BERTopic(seed_topic_list = seed_topic_list, verbose = True, nr_topics=20)

#train the model given the above info. 
topics, probs = topic_model.fit_transform(docs)
# topics_per_class = topic_model.topics_per_class(docs, classes = classes)




In [ ]:
#get topics groupings from model
topic_model.get_topic_info()

In [ ]:
#get topics from model and get corresponding topics = see relationship over different levels of granularity of topics.
hierarchical_topics = topic_model.hierarchical_topics(docs)

In [ ]:
#createh ierarchy of topics ; what sub topics are within the major 20 topics?

topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
#visualize hierarchy of topics 

tree = topic_model.get_topic_tree(hierarchical_topics)
tree

In [ ]:
#visualize topics in barchart
topic_model.visualize_barchart()

In [ ]:
#display topic goup 9
topic_model.get_topic(9)

In [ ]:
topic_model.visualize_topics()


--------------------------------------------------------
MISC
--------------------------------------------------------

In [ ]:
# This function takes a word and adds it to the txt file known words for spell checker to reference
   
# def add_known_word(word):
#         with open("known_words.txt", "a") as file:
#             file.write(word + "\n")
        